# VacationPy
----

In [2]:
#dependencies and setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from config import gkey

### Store Part I results into DataFrame

In [3]:
#create dataframe of city weather from csv
city_weather_df = pd.read_csv("output/citiesweather.csv", index_col = 0)

#preview frame
city_weather_df.head()

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,New Norfolk,-42.7826,147.0587,48.76,91,86,2.24,AU,1624321457
1,Salalah,17.0151,54.0924,78.89,94,90,5.75,OM,1624321457
2,Barrow,71.2906,-156.7887,41.02,75,1,9.22,US,1624321202
3,Atuona,-9.8000,-139.0333,79.21,76,2,12.28,PF,1624321384
4,Saskylakh,71.9167,114.0833,78.44,41,12,1.50,RU,1624321458


### Humidity Heatmap

In [4]:
#configure gmaps
gmaps.configure(api_key=gkey)

#customize the size of the figure
figure_layout = {
    'width': '1000px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

#define locations of interest and level of humidity
locations = city_weather_df[["Latitude", "Longitude"]]
humid = city_weather_df["Humidity"]

#build heatlayer
humidity_layer = gmaps.heatmap_layer(locations, weights = humid, 
                                 dissipating = True, max_intensity = 100,
                                 point_radius = 15)

#add heatlayer layer to map
fig.add_layer(humidity_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria

In [5]:
#create dataframe of cities that fit weather criteria
ideal_weather_df = city_weather_df.loc[(city_weather_df['Max Temperature'] > 70) & (city_weather_df['Max Temperature'] < 80) & (city_weather_df['Wind Speed'] < 10) & (city_weather_df['Cloudiness'] == 0),  :]

#view frame
ideal_weather_df

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
61,Alexandria,31.2156,29.9553,75.25,74,0,3.76,EG,1624321263
216,Belgrade,44.8040,20.4651,76.66,69,0,5.75,RS,1624321597
270,Togitsu,32.8333,129.8500,75.16,78,0,4.61,JP,1624321862
327,Seoul,37.5683,126.9778,76.57,62,0,3.44,KR,1624321669
377,Al Bardīyah,31.7561,25.0865,73.60,80,0,7.47,LY,1624322090
417,Kununurra,-15.7667,128.7333,71.58,40,0,3.44,AU,1624321989
446,Porteirinha,-15.7433,-43.0283,70.05,42,0,6.42,BR,1624322178
504,Portoferraio,42.8115,10.3146,73.53,86,0,9.57,IT,1624322261
508,Hambantota,6.1241,81.1185,77.00,100,0,4.61,LK,1624322262
530,Tūkrah,32.5341,20.5791,76.62,53,0,0.51,LY,1624322223


## Hotel Map

In [6]:
#store into new dataframe named hotel_df
hotel_df = ideal_weather_df

#create hotel name column within dataframe
hotel_df['Hotel Name'] = ''

#set parameters to search for hotels with 5000 meters.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = { 
    "radius": 5000,
    "type": "lodging",
    "keyword": "hotel",
    "key": gkey,
}
#iterate through pandas dataframe
for index, row in hotel_df.iterrows():

#define city and location
    lat = row['Latitude']
    lng = row['Longitude']
    city = row['City']
    
#set location variable within call specifications
    target_coordinates = f'{lat}, {lng}'

#add location parameter
    params['location'] = target_coordinates

    print(f"Retrieving Results for {city}.")

#assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
#extract results
    results = response['results']
    
#add found hotel to dataframe
    try:
        print(f"Closest hotel to {city} is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
#add 'none' to dataframe, if no hotel found or error
    except (KeyError, IndexError):
        hotel_df.loc[index, 'Hotel Name'] = "None"

        print("Missing field/result or no hotel within range.")
        
    print("------------")

C:\Users\terre\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
Retrieving Results for Alexandria.
Closest hotel to Alexandria is Four Seasons Hotel Alexandria at San Stefano, Egypt.
------------
Retrieving Results for Belgrade.
C:\Users\terre\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
Closest hotel to Belgrade is Falkensteiner Hotel Belgrade.
-

In [7]:
#create new dataframe of only cities with hotels withing 5000m of coordinates
hotel_clean_df = hotel_df.loc[hotel_df['Hotel Name'] != 'None', :]
hotel_clean_df

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
61,Alexandria,31.2156,29.9553,75.25,74,0,3.76,EG,1624321263,"Four Seasons Hotel Alexandria at San Stefano, ..."
216,Belgrade,44.8040,20.4651,76.66,69,0,5.75,RS,1624321597,Falkensteiner Hotel Belgrade
270,Togitsu,32.8333,129.8500,75.16,78,0,4.61,JP,1624321862,Yasuda Ocean Hotel Togitsu
327,Seoul,37.5683,126.9778,76.57,62,0,3.44,KR,1624321669,Grand Hyatt Seoul
377,Al Bardīyah,31.7561,25.0865,73.60,80,0,7.47,LY,1624322090,Hotel Burdy
417,Kununurra,-15.7667,128.7333,71.58,40,0,3.44,AU,1624321989,The Kimberley Grande Resort
446,Porteirinha,-15.7433,-43.0283,70.05,42,0,6.42,BR,1624322178,Portal Palace Hotel
504,Portoferraio,42.8115,10.3146,73.53,86,0,9.57,IT,1624322261,Hotel Hermitage
530,Tūkrah,32.5341,20.5791,76.62,53,0,0.51,LY,1624322223,مصيف مرحبا للعائلات
546,Tabas,33.5959,56.9244,75.43,11,0,3.15,IR,1624322338,هتل


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_clean_info = [info_box_template.format(**row) for index, row in hotel_clean_df.iterrows()]
locations = hotel_clean_df[["Latitude", "Longitude"]]

In [9]:
#build marker layer
markers = gmaps.marker_layer(locations, info_box_content= hotel_clean_info)

#add layer to figure with heatmap
fig.add_layer(markers)

#display
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…